In [ ]:
import pandas as pd

In [ ]:
import nltk, string
tokenizer = nltk.WordPunctTokenizer()

In [ ]:
import torch
import transformers 
import numpy as np

In [ ]:
tokenizer = transformers.BertTokenizer(
    vocab_file='ru_conversational_cased_L-12_H-768_A-12_pt/vocab.txt') 

In [ ]:
from tqdm import notebook

In [ ]:
config = transformers.BertConfig.from_json_file(
    'ru_conversational_cased_L-12_H-768_A-12_pt/bert_config.json')
model = transformers.BertModel.from_pretrained(
    'ru_conversational_cased_L-12_H-768_A-12_pt/pytorch_model.bin', config=config)

In [ ]:
marked = pd.read_excel('/content/drive/My Drive/marked.xlsx', index_col=0).dropna().reset_index(drop=True)
marked

In [ ]:
data = pd.read_excel('/content/drive/My Drive/marked.xlsx', index_col=0).reset_index(drop=True).loc[:500]
data['flood'] = 0
data['mark'] = data['mark'].fillna(2)
data.loc[data['mark']==2, 'flood'] = 1
data.loc[data['mark']==2, 'mark'] = np.nan

In [ ]:
tokenizer_ = nltk.WordPunctTokenizer()
def clear_text(raw):
  line = tokenizer_.tokenize(raw.lower())
  filtered_line = [w for w in line if all(c not in string.punctuation+''.join([str(x) for x in range(0, 10)]) for c in w) and len(w) > 3]
  return ' '.join(filtered_line)
marked['cl_text'] = marked['text'].apply(clear_text)

In [ ]:
tokenized = marked['cl_text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True))

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

In [ ]:
batch_size = 4
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

In [ ]:
mark_embs = np.concatenate(embeddings)
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(mark_embs, marked['mark'])


In [ ]:
from tensorflow import keras

net = keras.models.Sequential()


net.add(keras.layers.BatchNormalization(input_dim=768))
net.add(keras.layers.Dense(units=768*2,  activation='relu'))
net.add(keras.layers.Dropout(0.5))
net.add(keras.layers.BatchNormalization(input_dim=768))
net.add(keras.layers.Dropout(0.5))
net.add(keras.layers.Dense(units=2,  activation='softmax'))

net.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
net.fit(features_train, target_train, epochs=34, verbose=2,
          validation_data=(features_test, target_test))

In [ ]:
preds = [0 if x>y else 1 for x, y in net(features_test).numpy()]
f1_score(target_test, preds)

0.7499999999999999